# Simple LeNet for MNIST classification

Inspired by Yann LeCunns LeNet-5 (Y. LeCun, L. Bottou, Y. Bengio, and P. Haffner. Gradient-based learning applied to document recognition. Proceedings of the IEEE, 1998).

In [1]:
using Knet
using NNHelferlein
using MLDatasets: MNIST

### Get MNIST data from MLDatasets:

In [2]:
mnist_dir = joinpath(NNHelferlein.DATA_DIR, "mnist")
xtrn,ytrn = MNIST.traindata(Float32, dir=mnist_dir)
ytrn[ytrn.==0] .= 10
dtrn = minibatch(xtrn, ytrn, 128; xsize = (28,28,1,:))

xtst,ytst = MNIST.testdata(Float32, dir=mnist_dir)
ytst[ytst.==0] .= 10
dtst = minibatch(xtst, ytst, 128; xsize = (28,28,1,:));

### Define LeNet with NNHelferlein types:

In [3]:
lenet = Classifier(Conv(5,5,1,20),       
                Pool(),
                Conv(5,5,20,50),
                Pool(),
                Flat(),
                Dense(800,512), 
                Dense(512,10, actf=identity)
        )

Classifier((Conv(P(KnetArray{Float32, 4}(5,5,1,20)), P(KnetArray{Float32, 4}(1,1,20,1)), Knet.Ops20.relu, Base.Pairs{Symbol, Union{}, Tuple{}, NamedTuple{(), Tuple{}}}()), Pool(Base.Pairs{Symbol, Union{}, Tuple{}, NamedTuple{(), Tuple{}}}()), Conv(P(KnetArray{Float32, 4}(5,5,20,50)), P(KnetArray{Float32, 4}(1,1,50,1)), Knet.Ops20.relu, Base.Pairs{Symbol, Union{}, Tuple{}, NamedTuple{(), Tuple{}}}()), Pool(Base.Pairs{Symbol, Union{}, Tuple{}, NamedTuple{(), Tuple{}}}()), Flat(), Dense(P(Knet.KnetArrays.KnetMatrix{Float32}(512,800)), P(Knet.KnetArrays.KnetVector{Float32}(512)), Knet.Ops20.sigm), Dense(P(Knet.KnetArrays.KnetMatrix{Float32}(10,512)), P(Knet.KnetArrays.KnetVector{Float32}(10)), identity)))

In [4]:
print_network(lenet)

Neural network summary:
Classifier with 7 layers,                                       440812 params
Details:
 
    Conv layer 1 → 20 (5,5) with relu,                             520 params
    Pool layer,                                                      0 params
    Conv layer 20 → 50 (5,5) with relu,                          25050 params
    Pool layer,                                                      0 params
    Flat layer,                                                      0 params
    Dense layer 800 → 512 with sigm,                            410112 params
    Dense layer 512 → 10 with identity,                           5130 params
 
Total number of layers: 7
Total number of parameters: 440812


7

### Train with Tensorboard logger:

In [5]:
tb_train!(lenet, Adam, dtrn, epochs=2, split=0.8,
        acc_fun=accuracy,
        eval_size=0.2, eval_freq=5, mb_loss_freq=100, 
        tb_name="example_run", tb_text="NNHelferlein example")

println("Test loss:           $(lenet(dtst))")
println("Test accuracy:       $(accuracy(lenet, data=dtst))");

Splitting dataset for training (80%) and validation (20%).
Training 2 epochs with 374 minibatches/epoch and 94 validation mbs.
Evaluation is performed every 75 minibatches with 19 mbs.
Watch the progress with TensorBoard at:
/home/andreas/.julia/dev/NNHelferlein/examples/logs/example_run/2022-01-23T16-16-44


Progress: 100%|█████████████████████████████████████████| Time: 0:00:13


Training finished with:
Training loss:       0.06485884927990124
Training accuracy:   0.9804478609625669
Validation loss:     0.08135789355382006
Validation accuracy: 0.975814494680851
Test loss:           0.0653717
Test accuracy:       0.9805689102564102


Tensorboard output:    
<img src="assets/10-mnist-tb.png">